In [1]:
#이미지 변환
from PIL import Image
import piexif
from piexif import helper
import datetime
import os
import xml.etree.ElementTree as ET
from io import BytesIO

# 이미지 폴더 경로
image_folder = "/Users/a/Desktop/Work/Blog/2024 Adsense/wp/images/final/"

# 지원하는 이미지 확장자
supported_extensions = ['.jpg', '.jpeg', '.png', '.gif', '.bmp']

# 변환 결과 통계
total_files = 0
converted_files = 0
total_original_size = 0
total_jpeg_size = 0
total_webp_size = 0

# XMP 메타데이터 생성 함수
def create_xmp_metadata(keyword):
    # XMP 네임스페이스 정의
    xmp_ns = {
        'x': 'adobe:ns:meta/',
        'rdf': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#',
        'xmp': 'http://ns.adobe.com/xap/1.0/',
        'dc': 'http://purl.org/dc/elements/1.1/',
        'photoshop': 'http://ns.adobe.com/photoshop/1.0/',
    }
    
    # 현재 날짜/시간
    current_datetime = datetime.datetime.now().strftime("%Y-%m-%dT%H:%M:%S")
    
    # XMP 루트 요소 생성
    root = ET.Element('{' + xmp_ns['x'] + '}xmpmeta')
    for prefix, uri in xmp_ns.items():
        root.set(f'xmlns:{prefix}', uri)
    
    # RDF 요소 추가
    rdf = ET.SubElement(root, '{' + xmp_ns['rdf'] + '}RDF')
    
    # Description 요소 추가
    desc = ET.SubElement(rdf, '{' + xmp_ns['rdf'] + '}Description')
    desc.set('{' + xmp_ns['dc'] + '}title', keyword)
    desc.set('{' + xmp_ns['dc'] + '}description', f"{keyword}의 공식 배너 이미지")
    desc.set('{' + xmp_ns['xmp'] + '}CreatorTool', f"{keyword}_웹사이트")
    desc.set('{' + xmp_ns['xmp'] + '}CreateDate', current_datetime)
    desc.set('{' + xmp_ns['photoshop'] + '}Headline', keyword)
    
    # XML을 문자열로 변환
    xmp_str = ET.tostring(root, encoding='utf-8')
    return xmp_str

# 폴더 내 모든 파일 처리
for filename in os.listdir(image_folder):
    file_path = os.path.join(image_folder, filename)
    
    # 파일이 아니거나 지원하는 이미지 확장자가 아니면 건너뛰기
    if not os.path.isfile(file_path):
        continue
        
    file_ext = os.path.splitext(filename)[1].lower()
    if file_ext not in supported_extensions and file_ext != '.webp':
        continue
    
    total_files += 1
    
    try:
        # 파일 이름에서 키워드 생성 (확장자 제외)
        file_name = os.path.splitext(filename)[0]
        keyword = file_name.replace('-', ' ').replace('_', ' ')
        
        # 현재 날짜 시간
        current_datetime = datetime.datetime.now().strftime("%Y:%m:%d %H:%M:%S")
        
        # 이미지 열기
        img = Image.open(file_path)
        
        # 원본 파일 크기
        original_size = os.path.getsize(file_path) / 1024  # KB로 변환
        total_original_size += original_size
        
        # 메타데이터 준비 (JPEG용)
        zeroth_ifd = {
            piexif.ImageIFD.Make: keyword.encode('utf-8'),
            piexif.ImageIFD.XResolution: (300, 1),
            piexif.ImageIFD.YResolution: (300, 1),
            piexif.ImageIFD.Software: f"{keyword}_웹사이트".encode('utf-8')
        }
        
        # UserComment 생성
        comment_text = f"{keyword}의 공식 배너 이미지"
        user_comment = helper.UserComment.dump(comment_text, "unicode")
        
        exif_ifd = {
            piexif.ExifIFD.DateTimeOriginal: current_datetime.encode('utf-8'),
            piexif.ExifIFD.LensMake: keyword.encode('utf-8'),
            piexif.ExifIFD.UserComment: user_comment
        }
        
        # EXIF 데이터 생성
        exif_dict = {"0th": zeroth_ifd, "Exif": exif_ifd}
        exif_bytes = piexif.dump(exif_dict)
        
        # WebP용 XMP 메타데이터 생성
        xmp_metadata = create_xmp_metadata(keyword)
        
        # 파일명 생성 (JPG 버전)
        new_filename = "-".join(file_name.split())
        jpeg_output_path = os.path.join(image_folder, new_filename + ".jpg")
        
        # JPEG으로 저장 (EXIF 메타데이터 포함)
        img.save(jpeg_output_path, format="JPEG", quality=85, exif=exif_bytes)
        
        # WebP 파일명 생성
        webp_output_path = os.path.join(image_folder, new_filename + ".webp")
        
        # WebP 형식으로 변환하여 저장 (XMP 메타데이터 포함)
        # WebP 라이브러리는 XMP 메타데이터를 직접 지원하지 않으므로 부가 정보로 저장
        img.save(webp_output_path, format="WebP", quality=85, method=6, 
                 exif=exif_bytes,  # 일부 WebP 구현에서는 EXIF를 부분적으로 지원할 수 있음
                 xmp=xmp_metadata)  # XMP 메타데이터 추가 (PIL 1.9.0 이상에서 지원)
        
        # 파일 크기 계산
        jpeg_size = os.path.getsize(jpeg_output_path) / 1024  # KB로 변환
        webp_size = os.path.getsize(webp_output_path) / 1024  # KB로 변환
        
        total_jpeg_size += jpeg_size
        total_webp_size += webp_size
        
        print(f"이미지 처리 완료: {filename}")
        print(f"- 키워드: {keyword}")
        print(f"- 원본 파일 크기: {original_size:.2f} KB")
        print(f"- JPEG 파일 크기: {jpeg_size:.2f} KB")
        print(f"- WebP 파일 크기: {webp_size:.2f} KB")
        print(f"- JPEG 대비 WebP 절약: {jpeg_size - webp_size:.2f} KB ({(1 - webp_size/jpeg_size) * 100:.1f}% 감소)")
        print("-" * 50)
        
        converted_files += 1
        
    except Exception as e:
        print(f"오류 발생: {filename} - {str(e)}")

# 최종 결과 출력
print("\n======= 변환 완료 요약 =======")
print(f"총 이미지 파일: {total_files}")
print(f"변환 성공: {converted_files}")
print(f"변환 실패: {total_files - converted_files}")
print(f"총 원본 크기: {total_original_size:.2f} KB")
print(f"총 JPEG 크기: {total_jpeg_size:.2f} KB")
print(f"총 WebP 크기: {total_webp_size:.2f} KB")
print(f"총 절약된 용량(JPEG 대비): {total_jpeg_size - total_webp_size:.2f} KB ({(1 - total_webp_size/total_jpeg_size) * 100:.1f}% 감소)")

이미지 처리 완료: 대표-기도문-나눔터-모음.jpg
- 키워드: 대표 기도문 나눔터 모음
- 원본 파일 크기: 29.08 KB
- JPEG 파일 크기: 24.53 KB
- WebP 파일 크기: 11.46 KB
- JPEG 대비 WebP 절약: 13.07 KB (53.3% 감소)
--------------------------------------------------
이미지 처리 완료: 쉬운-대표-기도문-모음.jpg
- 키워드: 쉬운 대표 기도문 모음
- 원본 파일 크기: 47.90 KB
- JPEG 파일 크기: 40.46 KB
- WebP 파일 크기: 24.01 KB
- JPEG 대비 WebP 절약: 16.46 KB (40.7% 감소)
--------------------------------------------------
이미지 처리 완료: 짧은-대표-기도문-모음.jpg
- 키워드: 짧은 대표 기도문 모음
- 원본 파일 크기: 23.81 KB
- JPEG 파일 크기: 20.89 KB
- WebP 파일 크기: 9.44 KB
- JPEG 대비 WebP 절약: 11.45 KB (54.8% 감소)
--------------------------------------------------
이미지 처리 완료: 주일-대표-기도문-모음.jpg
- 키워드: 주일 대표 기도문 모음
- 원본 파일 크기: 36.87 KB
- JPEG 파일 크기: 31.73 KB
- WebP 파일 크기: 15.69 KB
- JPEG 대비 WebP 절약: 16.03 KB (50.5% 감소)
--------------------------------------------------

======= 변환 완료 요약 =======
총 이미지 파일: 4
변환 성공: 4
변환 실패: 0
총 원본 크기: 137.66 KB
총 JP

In [23]:
#변환 확인
from PIL import Image
from PIL.ExifTags import TAGS
import piexif
import os

def get_image_metadata(image_path):
    """이미지 파일의 모든 메타데이터를 추출하고 출력하는 함수"""
    if not os.path.exists(image_path):
        print(f"오류: 파일을 찾을 수 없습니다: {image_path}")
        return
    
    print(f"파일: {os.path.basename(image_path)}")
    print("-" * 50)
    
    try:
        # 이미지 열기
        img = Image.open(image_path)
        
        # 기본 이미지 정보
        print("[ 기본 이미지 정보 ]")
        print(f"포맷: {img.format}")
        print(f"모드: {img.mode}")
        print(f"크기: {img.size[0]} x {img.size[1]} 픽셀")
        print(f"파일 크기: {os.path.getsize(image_path) / 1024:.2f} KB")
        print()
        
        # EXIF 데이터 추출
        print("[ EXIF 데이터 ]")
        exif_data_extracted = False
        
        # 방법 1: PIL의 _getexif 메서드 사용
        if hasattr(img, '_getexif') and img._getexif():
            exif_info = img._getexif()
            if exif_info:
                for tag, value in exif_info.items():
                    decoded = TAGS.get(tag, tag)
                    if isinstance(value, bytes):
                        try:
                            value = value.decode('utf-8')
                        except UnicodeDecodeError:
                            value = str(value)
                    print(f"{decoded}: {value}")
                    exif_data_extracted = True
        
        # 방법 2: piexif 라이브러리 사용
        try:
            exif_dict = piexif.load(img.info.get('exif', b''))
            for ifd_name in exif_dict:
                if ifd_name != 'thumbnail' and exif_dict[ifd_name]:
                    if not exif_data_extracted:
                        exif_data_extracted = True
                    
                    if isinstance(exif_dict[ifd_name], dict):  # IFD 타입 확인
                        for tag_id in exif_dict[ifd_name]:
                            try:
                                if ifd_name in piexif.TAGS and tag_id in piexif.TAGS[ifd_name]:
                                    tag_name = piexif.TAGS[ifd_name][tag_id]["name"]
                                    tag_value = exif_dict[ifd_name][tag_id]
                                    
                                    if isinstance(tag_value, bytes):
                                        try:
                                            tag_value = tag_value.decode('utf-8')
                                        except UnicodeDecodeError:
                                            tag_value = str(tag_value)
                                    
                                    print(f"{tag_name}: {tag_value}")
                            except:
                                pass
        except Exception as e:
            if not exif_data_extracted:
                print(f"EXIF 데이터 추출 오류: {e}")
        
        if not exif_data_extracted:
            print("EXIF 데이터가 없거나 추출할 수 없습니다.")
        
        # 기타 메타데이터 (이미지 정보)
        print("\n[ 기타 메타데이터 ]")
        other_info = False
        for key, value in img.info.items():
            if key != 'exif':  # EXIF 데이터는 이미 처리했으므로 제외
                if isinstance(value, bytes):
                    try:
                        value = value.decode('utf-8')
                    except UnicodeDecodeError:
                        value = str(value)
                print(f"{key}: {value}")
                other_info = True
        
        if not other_info:
            print("추가 메타데이터가 없습니다.")
        
    except Exception as e:
        print(f"이미지 처리 중 오류 발생: {e}")
    
    print("-" * 50)

# 메인 코드
image_path = "/Users/a/Desktop/Work/Blog/2024 Adsense/wp/images/final/주일-대표-기도문-모음.webp"
get_image_metadata(image_path)

파일: 주일-대표-기도문-모음.webp
--------------------------------------------------
[ 기본 이미지 정보 ]
포맷: WEBP
모드: RGB
크기: 640 x 427 픽셀
파일 크기: 14.80 KB

[ EXIF 데이터 ]
Software: áá®ááµá¯ áá¢áá­ ááµáá©áá®á« áá©áá³á·_ì¹ì¬ì´í¸
ExifOffset: 237
XResolution: 300.0
YResolution: 300.0
Make: áá®ááµá¯ áá¢áá­ ááµáá©áá®á« áá©áá³á·
LensMake: áá®ááµá¯ áá¢áá­ ááµáá©áá®á« áá©áá³á·
DateTimeOriginal: 2025:05:07 16:42:48
UserComment: b'UNICODE\x00\x11\x0c\x11n\x11\x0b\x11u\x11\xaf\x00 \x11\x03\x11b\x11\x11\x11m\x00 \x11\x00\x11u\x11\x03\x11i\x11\x06\x11n\x11\xab\x00 \x11\x06\x11i\x11\x0b\x11s\x11\xb7\xc7X\x00 \xac\xf5\xc2\xdd\x00 \xbc0\xb1\x08\x00 \xc7t\xbb\xf8\xc9\xc0'
Make: 주일 대표 기도문 모음
XResolution: (300, 1)
YResolution: (300, 1)
Software: 주일 대표 기도문 모음_웹사이트
ExifTag: 237
DateTimeOriginal: 2025:05:07 16:42:48
UserComment: b'UNICODE\x00\x11\x0c\x11n\x11\x0b\x11u\x11\xaf\x00 \x11\x03\x11b\x11\x1